In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime
import pydicom
from pydicom.valuerep import DT
from pydicom.sr.codedict import codes
import highdicom as hd
import structured_reports as sr

In [3]:
ecg = pydicom.dcmread('example_ecg.dcm')

In [4]:
procedure_datetime = DT(datetime.now().strftime("%Y%m%d%H%M%S.%f"))
ecg_concept_name = hd.sr.CodedConcept(
    value='C0018799', 
    scheme_designator='SCT', 
    meaning='Electrocardiogram'
)

referenced_sop_class_uid = hd.UID('1.2.840.10008.5.1.4.1.1.9.1.1')  # Example: Waveform Storage SOP Class
referenced_sop_instance_uid = ecg.SOPInstanceUID

# https://dicom.nema.org/medical/dicom/current/output/chtml/part16/sect_TID_3708.html
source_of_measurement = hd.sr.WaveformContentItem(
    name=ecg_concept_name,
    referenced_sop_class_uid=referenced_sop_class_uid, 
    referenced_sop_instance_uid=referenced_sop_instance_uid, 
    referenced_waveform_channels=None, 
    relationship_type=hd.sr.RelationshipTypeValues.CONTAINS
)

# https://dicom.nema.org/medical/dicom/current/output/chtml/part16/sect_CID_3263.html
lead_system = hd.sr.CodedConcept(
    value='10:11265',
    scheme_designator='MDC',
    meaning='Standard 12-lead positions, electrodes placed individually'
)

ecg_waveform_information = sr.ECGWaveFormInformation(
    procedure_datetime=procedure_datetime,
    source_of_measurement=source_of_measurement,
    lead_system=lead_system,
    acquisition_device_type='Electrocardiographic monitor and recorder',
    equipment_identification=ecg.Manufacturer,
)

In [5]:
language = hd.sr.CodedConcept(
    value='de-DE',
    scheme_designator='RFC5646',
    meaning='German (Germany)'
)

observer_person_context = hd.sr.ObserverContext(
    observer_type=codes.DCM.Person,
    observer_identifying_attributes=hd.sr.PersonObserverIdentifyingAttributes(
        name=ecg.PatientName
    )
)
observer_device_context = hd.sr.ObserverContext(
    observer_type=codes.DCM.Device,
    observer_identifying_attributes=hd.sr.DeviceObserverIdentifyingAttributes(
        uid=hd.UID()
    )
)

ecg_global_measurements = sr.ECGGlobalMeasurements(
    ventricular_heart_rate= 75.0,
    qt_interval_global=400.0,
    pr_interval_global=140.0,
    qrs_duration_global=100.0,
    rr_interval_global=800.0,
)

quantitatitative_analysis = sr.QuantitativeAnalysis(
    ecg_global_measurements=ecg_global_measurements
)

In [6]:
patient_characteristics_for_ecg = sr.PatientCharacteristicsForECG(
    subject_age=int(ecg.PatientAge[:-1]), # format: e.g. 060Y
    subject_sex=ecg.PatientSex,
    systolic_blood_pressure=160.0,
    diastolic_blood_pressure=100.0,
    patient_state=hd.sr.CodedConcept(
        value='128975004',
        scheme_designator='SCT',
        meaning='Resting state'
    ),
    pacemaker_in_situ=hd.sr.CodedConcept(
        value='D',
        scheme_designator='NBG',
        meaning='Dual (A+V)'
    )
)

In [7]:
history_of_myocaridal_infarction = sr.ProblemProperties(
    concern_type=hd.sr.CodeContentItem(
        name=hd.sr.CodedConcept(
            value='418799008',
            scheme_designator='SCT',
            meaning='Finding reported by patient/informant'
        ),
        value=hd.sr.CodedConcept(
            value='399211009',
            scheme_designator='SCT',
            meaning='History of myocardidal infarction'
        ),
        relationship_type=hd.sr.RelationshipTypeValues.CONTAINS
    ),
    datetime_concern_noted=DT(datetime.now().strftime("%Y%m%d%H%M%S.%f")),
    therapies=[
        sr.Therapy(
            name=hd.sr.CodedConcept(
                value='232717009',
                scheme_designator='SCT',
                meaning='Coronary artery bypass graft'
            )
        )
    ]
)

cardiac_patient_risk_factors = [
    sr.ProblemProperties(
        concern_type=hd.sr.CodeContentItem(
            name=hd.sr.CodedConcept(
                value='418799008',
                scheme_designator='SCT',
                meaning='Finding reported by patient/informant'
            ),
            value=hd.sr.CodedConcept(
                value='400047006',
                scheme_designator='SCT',
                meaning='Peripheral vascular disease'
            ),
            relationship_type=hd.sr.RelationshipTypeValues.CONTAINS
        ),
        datetime_concern_noted=DT(datetime.now().strftime("%Y%m%d%H%M%S.%f"))
    )
]

problem_list = sr.ProblemList(
    cardiac_patient_risk_factors=cardiac_patient_risk_factors,
    history_of_myocardial_infarction=history_of_myocaridal_infarction
)

social_history = sr.SocialHistory(
    tobacco_smoking_behavior=hd.sr.CodedConcept(
        value='77176002',
        scheme_designator='SCT',
        meaning='Current Smoker'
    )
)

past_surgical_history = sr.PastSurgicalHistory(
    procedure_properties=[
        sr.ProcedureProperties(
            name=hd.sr.CodedConcept(
                value='387713003', 
                scheme_designator='SCT',
                meaning='Surgical Procedure'
            ),
            value=hd.sr.CodedConcept(
                value='232717009',
                scheme_designator='SCT',
                meaning='Coronary artery bypass graft'
            )
        ),
        sr.ProcedureProperties(
            name=hd.sr.CodedConcept(
                value='387713003', 
                scheme_designator='SCT',
                meaning='Surgical Procedure'
            ),
            value=hd.sr.CodedConcept(
                value='307280005',
                scheme_designator='SCT',
                meaning='Implantation of cardiac pacemaker'
            )
        )
    ]
)

relevant_diagnostic_tests_and_or_laboratory_data = sr.RelevantDiagnosticTestsAndOrLaboratoryData(
    cholesterol_in_HDL=75.0,
    cholesterol_in_LDL=160.0
)

cardiovascular_patient_history = sr.CardiovascularPatientHistory(
    problem_list=problem_list,
    social_history=social_history,
    past_surgical_history=past_surgical_history,
    relevant_diagnostic_tests_and_or_laboratory_data=relevant_diagnostic_tests_and_or_laboratory_data,
)

In [8]:
ecg_report = sr.ECGReport(
    language_of_content_item_and_descendants=hd.sr.LanguageOfContentItemAndDescendants(language),
    observer_contexts=[observer_person_context, observer_device_context],
    ecg_waveform_information=ecg_waveform_information,
    quantitative_analysis=quantitatitative_analysis,
    cardiovascular_patient_history=cardiovascular_patient_history,
    patient_characteristics_for_ecg=patient_characteristics_for_ecg,
)

In [9]:
ecg_sr = hd.sr.ComprehensiveSR(
    evidence=[ecg],
    content=ecg_report,
    series_number=1,
    series_instance_uid=hd.UID(),
    sop_instance_uid=hd.UID(),
    instance_number=1
)
ecg_sr

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 226
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: Comprehensive SR Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.2.826.0.1.3680043.10.511.3.56431750657457273228940717842343408
(0002, 0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.2.826.0.1.3680043.9.7433.1.1
(0002, 0013) Implementation Version Name         SH: 'highdicom0.21.1'
-------------------------------------------------
(0008, 0016) SOP Class UID                       UI: Comprehensive SR Storage
(0008, 0018) SOP Instance UID                    UI: 1.2.826.0.1.3680043.10.511.3.56431750657457273228940717842343408
(0008, 0020) Study Date                          DA: '20240101'
(0008, 0023) Content Date                        DA: "20240902"
(0008, 0030) Study Time 

In [10]:
ecg_sr.save_as('example_ecg_sr.dcm')